In [1]:
import numpy as np
import xarray as xr
import pandas as pd 
import sys 
from EchoPro import EchoPro
epro_2019 = EchoPro(init_file_path='./config_files/initialization_config.yml',
                    survey_year_file_path='./config_files/survey_year_2019_config.yml',
                    source=3,
                    bio_data_type=1,
                    age_data_status=1, 
                    exclude_age1=True)

A check of the initialization file needs to be done!
A check of the survey year file needs to be done!
Loading biological data ...


/Users/brandonreyes/UW_work/EchoPro_work/EchoPro/EchoPro/echo_pro.py:269: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.final_biomass_table["wgt_total"] = mat['wgt_total']
/Users/brandonreyes/UW_work/EchoPro_work/EchoPro/EchoPro/echo_pro.py:271: UserWarning: We are currently using wgt_total from Matlab for CV, change this!
  warnings.warn("We are currently using wgt_total from Matlab for CV, change this!")


In [2]:
# epro_2019.final_biomass_table

In [7]:
if epro_2019.params["JH_fac"] == 1:
    nr = 1              # number of realizations
else:
    nr = 10000          # number of realizations

lat_INPFC = [np.NINF, 36, 40.5, 43.000, 45.7667, 48.5, 55.0000]  # INPFC
lon_inc = 0.05 # longitude increment along the transect

transect_info, strata_info = epro_2019.get_transect_strata_info(lat_INPFC)

strata_info

,count,sum
lat_INPFC_bin,,
0,10,4246.476768
1,24,10042.011575
2,13,5773.924105
3,15,7059.964585
4,15,7067.793322
5,36,19318.827007


In [75]:
import sys 


for bin_val in strata_info.index:
    
    num_ind = round(epro_2019.params["JH_fac"]*strata_info.loc[bin_val]['count'])
#     sel_ind = rng.choice(transect_info.loc[bin_val].index, num_ind, replace=False)
    sel_ind = np.array([1, 3, 4, 5, 6, 7, 9, 10])
    print(f"num_ind = {num_ind}")
    print(sel_ind)
    
    transect_bin_info = transect_info.loc[(bin_val, sel_ind), ['distance', 'biomass']]
    
#     print(transect_bin_info)
    
    # transect-length weighting factor of the transects in the "bin_val" bin
    wgt = transect_bin_info['distance']/transect_bin_info['distance'].mean()
    print(wgt)
    
    # normalized biomass of the transects in the "bin_val" bin
    rhom_trans_bin = transect_bin_info['biomass']/transect_bin_info['distance']
    print(rhom_trans_bin)
    
    # transect-length-normalized mean density of the "bin_val" bin
    rhom_trans = (transect_bin_info['biomass']*transect_bin_info['distance']).sum()/transect_bin_info['distance'].sum() #sum(biomass_ij.*Lij)/sum(Lij)
    
    print(rhom_trans)
    sys.exit()
    

num_ind = 8
[ 1  3  4  5  6  7  9 10]
lat_INPFC_bin  Transect
0              1           0.806847
               3           1.174273
               4           1.117556
               5           1.114505
               6           0.994281
               7           1.055282
               9           0.931624
               10          0.805632
Name: distance, dtype: float64
lat_INPFC_bin  Transect
0              1                0.000000
               3                0.000000
               4                0.000000
               5                0.000000
               6                0.000000
               7                0.000000
               9            40113.997421
               10          188899.711433
dtype: float64
848928.0039102882


SystemExit: 

/Users/brandonreyes/opt/anaconda3/envs/echopro_conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3369: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
